# Experiment_dataset

In [1]:
import pandas as pd
from dataset.data_manager import BuildDataset

from dataset.dataset_labeling_study import aggiorna_label_distanza_temporale
from dataset.dataset_labeling_study import create_tile_videos_relabeling, create_df_video_from_master_df_relabeling

/home/isac/miniconda3/envs/videomae/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# NON carico quello con tutti i metodi all_data_all_methods_tracks_complete_fast.csv

sup_data_train = BuildDataset(type='SUPERVISED', master_df_path="all_data_CL7_tracks_complete_fast.csv")
sup_data_train.load_master_df()

In [3]:
sup_data_train.master_df.dtypes

path             string[python]
datetime         datetime64[ns]
tile_offset_x             int16
tile_offset_y             int16
x_pix                    object
y_pix                    object
source           string[python]
id_cyc_unico              int32
start_time       datetime64[ns]
end_time         datetime64[ns]
label                      int8
dtype: object

In [4]:
sup_data_train.calc_delta_time()

In [5]:
sup_data_train.master_df

,path,datetime,tile_offset_x,tile_offset_y,x_pix,y_pix,source,id_cyc_unico,start_time,end_time,label,delta_time
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,0,[],[],[],0,NaT,NaT,0,NaT
1,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,213,0,[],[],[],0,NaT,NaT,0,NaT
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,426,0,[],[],[],0,NaT,NaT,0,NaT
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,639,0,[],[],[],0,NaT,NaT,0,NaT
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,852,0,[],[],[],0,NaT,NaT,0,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...
1413163,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,213,196,[],[],[],0,NaT,NaT,0,NaT
1413164,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,426,196,[],[],[],0,NaT,NaT,0,NaT
1413165,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,639,196,[],[],[],0,NaT,NaT,0,NaT
1413166,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,852,196,[],[],[],0,NaT,NaT,0,NaT


In [6]:
new_label = aggiorna_label_distanza_temporale(sup_data_train.master_df, soglia=pd.Timedelta(hours=12), sub_lab=-1)
new_label

Colonna label_12_00 aggiunta


'label_12_00'

In [ ]:
######################################
# # verifico che il relabeling sia corretto per la parte iniziale e finale della durata del ciclone
pd.set_option('display.max_rows', 500)
m = sup_data_train.master_df[new_label] == -1
moff = (sup_data_train.master_df[['tile_offset_x','tile_offset_y']] == (213,0)).all(axis=1)

m_cyc = sup_data_train.master_df.id_cyc_unico == 7001328
sup_data_train.master_df[m_cyc].loc[20545:23700] #14545:14600]

### ora levo le righe con label == -1 e poi prendo la colonna relabel come label ufficiale

In [7]:
m = sup_data_train.master_df[new_label] == -1
df_mod = sup_data_train.master_df[~m].copy().drop(columns='label').rename(columns={new_label:'label'})

In [8]:
df_mod

,path,datetime,tile_offset_x,tile_offset_y,x_pix,y_pix,source,id_cyc_unico,start_time,end_time,delta_time,label
0,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,0,0,[],[],[],0,NaT,NaT,NaT,0
1,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,213,0,[],[],[],0,NaT,NaT,NaT,0
2,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,426,0,[],[],[],0,NaT,NaT,NaT,0
3,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,639,0,[],[],[],0,NaT,NaT,NaT,0
4,../fromgcloud/airmass_rgb_20111101_0000.png,2011-11-01,852,0,[],[],[],0,NaT,NaT,NaT,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1413163,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,213,196,[],[],[],0,NaT,NaT,NaT,0
1413164,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,426,196,[],[],[],0,NaT,NaT,NaT,0
1413165,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,639,196,[],[],[],0,NaT,NaT,NaT,0
1413166,../fromgcloud/airmass_rgb_20230912_0000.png,2023-09-12,852,196,[],[],[],0,NaT,NaT,NaT,0


In [9]:
sup_data_train.make_df_video(new_master_df=df_mod, output_dir=None,  is_to_balance=True) #, idxs=[1,2,3,4,5,6,7,8])

1)  ->
Num video CON cicloni: 104, Num video SENZA cicloni: 2018	Totale video tiles: 2122		Bilanciamento video...
 video senza cicloni tenuti: 104
208 video per il periodo (effettivo) da 2011-11-01 00:00:00 a 2011-11-10 22:10:00

2)  ->
Num video CON cicloni: 99, Num video SENZA cicloni: 2016	Totale video tiles: 2115		Bilanciamento video...
 video senza cicloni tenuti: 99
198 video per il periodo (effettivo) da 2014-11-03 01:20:00 a 2014-11-12 23:55:00

3)  ->
Num video CON cicloni: 88, Num video SENZA cicloni: 1622	Totale video tiles: 1710		Bilanciamento video...
 video senza cicloni tenuti: 88
176 video per il periodo (effettivo) da 2016-10-26 00:00:00 a 2016-11-02 23:55:00

4)  ->
Num video CON cicloni: 83, Num video SENZA cicloni: 1618	Totale video tiles: 1701		Bilanciamento video...
 video senza cicloni tenuti: 83
166 video per il periodo (effettivo) da 2017-11-13 00:00:00 a 2017-11-20 15:55:00

5)  ->
Num video CON cicloni: 61, Num video SENZA cicloni: 998	Totale video tiles: 105

In [10]:
df_v = sup_data_train.df_video.copy()

In [11]:
print(df_v.label.sum(), df_v.shape[0])

1773 3546


In [17]:
train_p = 0.7
len_p = int(train_p*df_v.shape[0])
df_video_train = df_v.sort_values('start_time').iloc[:len_p]
df_video_test = df_v.sort_values('start_time').iloc[len_p:]
df_video_train.shape, df_video_test.shape

((2482, 8), (1064, 8))

# Split train e test dal file di Manos

In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
from dataset.data_manager import BuildDataset, get_intervals_in_tracks_df
from dataset.build_dataset import train_test_cyclones_num_split, get_train_test_df, get_train_test_validation_df
from medicane_utils.load_files import load_all_images_in_intervals, get_files_from_folder, extract_dates_pattern_airmass_rgb_20200101_0000, in_any_interval_via_index

import numpy as np
import pandas as pd
#from intervaltree import IntervalTree

from arguments import prepare_finetuning_args


input_dir="../fromgcloud"
output_dir = "../airmassRGB/supervised/" 

### costruisco il dataset dei cicloni 10

In [3]:
tracks_df_MED_CL10 = pd.read_csv("manos_CL10_pixel.csv", parse_dates=['time', 'start_time', 'end_time'])
tracks_df_MED_CL10.shape[0]

3397

In [4]:
# divido le track di Manos in train e test
cicloni_unici_train, cicloni_unici_test = train_test_cyclones_num_split(tracks_df_MED_CL10)

Cicloni nel train: 28, cicloni nel test: 12


In [5]:
tracks_df_train = tracks_df_MED_CL10[tracks_df_MED_CL10.id_cyc_unico.isin(cicloni_unici_train)]
tracks_df_test = tracks_df_MED_CL10[tracks_df_MED_CL10.id_cyc_unico.isin(cicloni_unici_test)]
print(f"Train e test lengths: {tracks_df_train.shape[0]}, {tracks_df_test.shape[0]}")

Train e test lengths: 2404, 993


In [6]:
# è come usare la fuzione "get_intervals_in_tracks_df"

u_train = tracks_df_train.groupby(tracks_df_train.id_cyc_unico).apply('first')  
print((u_train.end_time - u_train.start_time).sum())

u_test = tracks_df_test.groupby(tracks_df_test.id_cyc_unico).apply('first')  
print((u_test.end_time - u_test.start_time).sum())

105 days 22:00:00
47 days 03:00:00


In [21]:
train_m = BuildDataset(type='SUPERVISED')
train_m.get_data_ready(tracks_df_train, input_dir, output_dir, csv_file="train_CL10")

sorted_metadata_files num :  1285
Colonna label_12_00 aggiunta
1)  ->
Num video CON cicloni: 98, Num video SENZA cicloni: 1150	Totale video tiles: 1248		Bilanciamento video...
 video senza cicloni tenuti: 98



Creazione delle folder per i 196 video...	Salvati 3136 file - Erano già presenti 0 file - File totali 3136
196 video per il periodo (effettivo) da 2014-11-05 23:45:00 a 2014-11-10 07:00:00



In [18]:
train_m.master_df[['tile_offset_x', 'tile_offset_y']].value_counts()

tile_offset_x  tile_offset_y
0              0                1285
               196              1285
213            0                1285
               196              1285
426            0                1285
639            0                1285
               196              1285
852            0                1285
1065           0                1285
               196              1285
426            196              1141
852            196              1141
Name: count, dtype: int64

In [89]:
test_m = BuildDataset(type='SUPERVISED')
test_m.get_data_ready(tracks_df_test, input_dir, output_dir, csv_file="test_CL10")

sorted_metadata_files num :  6510
Colonna label_12_00 aggiunta
1)  ->
Num video CON cicloni: 37, Num video SENZA cicloni: 511	Totale video tiles: 548		Bilanciamento video...
 video senza cicloni tenuti: 37



Creazione delle folder per i 74 video...	Salvati 192 file - Erano già presenti 992 file - File totali 1184
74 video per il periodo (effettivo) da 2020-03-30 16:45:00 a 2020-04-01 14:00:00

2)  ->
Num video CON cicloni: 60, Num video SENZA cicloni: 849	Totale video tiles: 909		Bilanciamento video...
 video senza cicloni tenuti: 60



Creazione delle folder per i 120 video...	Salvati 1088 file - Erano già presenti 832 file - File totali 1920
120 video per il periodo (effettivo) da 2020-04-19 20:20:00 a 2020-04-23 00:00:00

3)  ->
Num video CON cicloni: 101, Num video SENZA cicloni: 1003	Totale video tiles: 1104		Bilanciamento video...
 video senza cicloni tenuti: 101



Creazione delle folder per i 202 video...	Salvati 1376 file - Erano già presenti 1856 file - File totali 3232
202 

In [90]:
train_len, test_len = train_m.df_video.shape[0], test_m.df_video.shape[0]
print(f"Training set: {train_len} samples - Test set: {test_len} samples")

Training set: 196 samples - Test set: 996 samples


### Costruisco dataset con solo Medicanes

In [4]:
tracks_df_medicanes = pd.read_csv("manos_medicanes_only.csv", parse_dates=['time', 'start_time', 'end_time'])
tracks_df_medicanes.shape[0]

878

In [12]:
more_medicanes = pd.read_csv("more_medicanes_time_updated.csv", parse_dates=['time', 'start_time', 'end_time'])
more_medicanes.shape[0]

2103

In [17]:
more_medicanes

,id_cyc_unico,lat,lon,time,source,x_pix,y_pix,idorig,name,start_time,end_time
0,7001283.0,36.300,7.220,2010-10-11 17:00:00,CL7,441,237,1283.0,NaN,2010-10-12 12:55:00,2010-10-13 13:25:00
1,7001283.0,36.479,6.902,2010-10-11 18:00:00,CL7,432,233,1283.0,NaN,2010-10-12 12:55:00,2010-10-13 13:25:00
2,7001283.0,36.655,6.649,2010-10-11 19:00:00,CL7,425,228,1283.0,NaN,2010-10-12 12:55:00,2010-10-13 13:25:00
3,7001283.0,36.844,6.496,2010-10-11 20:00:00,CL7,421,223,1283.0,NaN,2010-10-12 12:55:00,2010-10-13 13:25:00
4,7001283.0,37.040,6.375,2010-10-11 21:00:00,CL7,418,218,1283.0,NaN,2010-10-12 12:55:00,2010-10-13 13:25:00
...,...,...,...,...,...,...,...,...,...,...,...
2098,NaN,41.000,7.750,2023-03-03 04:00:00,ERA5,460,119,NaN,Juliette,2023-02-27 19:00:00,2023-03-02 13:30:00
2099,NaN,41.000,7.750,2023-03-03 05:00:00,ERA5,460,119,NaN,Juliette,2023-02-27 19:00:00,2023-03-02 13:30:00
2100,NaN,40.750,7.500,2023-03-03 06:00:00,ERA5,453,125,NaN,Juliette,2023-02-27 19:00:00,2023-03-02 13:30:00
2101,NaN,40.500,7.500,2023-03-03 07:00:00,ERA5,453,131,NaN,Juliette,2023-02-27 19:00:00,2023-03-02 13:30:00


In [23]:
from utils import str2num

mask_missing = more_medicanes['id_cyc_unico'].isna()
more_medicanes.loc[mask_missing, 'id_final'] = (
    more_medicanes.loc[mask_missing, 'name'].apply(str2num, digits=5)
)

# 2) ricopia gli id numerici già esistenti
more_medicanes.loc[~mask_missing, 'id_final'] = more_medicanes.loc[~mask_missing, 'id_cyc_unico']

# 3) assicura dtype Int64 (gestisce eventuali NA)
more_medicanes['id_final'] = more_medicanes['id_final'].astype('Int64')

In [26]:
more_medicanes

,id_cyc_unico,lat,lon,time,source,x_pix,y_pix,idorig,name,start_time,end_time,id_final
0,7001283.0,36.300,7.220,2010-10-11 17:00:00,CL7,441,237,1283.0,NaN,2010-10-12 12:55:00,2010-10-13 13:25:00,7001283
1,7001283.0,36.479,6.902,2010-10-11 18:00:00,CL7,432,233,1283.0,NaN,2010-10-12 12:55:00,2010-10-13 13:25:00,7001283
2,7001283.0,36.655,6.649,2010-10-11 19:00:00,CL7,425,228,1283.0,NaN,2010-10-12 12:55:00,2010-10-13 13:25:00,7001283
3,7001283.0,36.844,6.496,2010-10-11 20:00:00,CL7,421,223,1283.0,NaN,2010-10-12 12:55:00,2010-10-13 13:25:00,7001283
4,7001283.0,37.040,6.375,2010-10-11 21:00:00,CL7,418,218,1283.0,NaN,2010-10-12 12:55:00,2010-10-13 13:25:00,7001283
...,...,...,...,...,...,...,...,...,...,...,...,...
2098,NaN,41.000,7.750,2023-03-03 04:00:00,ERA5,460,119,NaN,Juliette,2023-02-27 19:00:00,2023-03-02 13:30:00,83565
2099,NaN,41.000,7.750,2023-03-03 05:00:00,ERA5,460,119,NaN,Juliette,2023-02-27 19:00:00,2023-03-02 13:30:00,83565
2100,NaN,40.750,7.500,2023-03-03 06:00:00,ERA5,453,125,NaN,Juliette,2023-02-27 19:00:00,2023-03-02 13:30:00,83565
2101,NaN,40.500,7.500,2023-03-03 07:00:00,ERA5,453,131,NaN,Juliette,2023-02-27 19:00:00,2023-03-02 13:30:00,83565


In [29]:
tracks_df_train, tracks_df_test, tracks_df_val = get_train_test_validation_df(more_medicanes, 0.7, 0.15)

Cicloni nel train: 13, cicloni nel test: 3, cicloni nella validation: 3
Train: 1307, Test: 433, Validation: 363
26 days 14:10:00
4 days 22:55:00
7 days 17:10:00


In [ ]:
args = prepare_finetuning_args()
train_m = BuildDataset(type='SUPERVISED', args=args)
train_m.get_data_ready(tracks_df_train, input_dir, output_dir, csv_file="train_medicanes")

sorted_metadata_files num :  15512
1)  ->
Num video CON cicloni: 72, Num video SENZA cicloni: 528	Totale video tiles: 600
Bilanciamento video...		 video senza cicloni tenuti: 72

Creazione delle folder per i 144 video...	Salvati 2304 file - Erano già presenti 0 file - File totali 2304
144 video per il periodo (effettivo) da 2010-10-11 17:45:00 a 2010-10-13 20:00:00

2)  ->
Num video CON cicloni: 115, Num video SENZA cicloni: 1133	Totale video tiles: 1248
Bilanciamento video...		 video senza cicloni tenuti: 115

Creazione delle folder per i 230 video...	Salvati 1647 file - Erano già presenti 2033 file - File totali 3680
230 video per il periodo (effettivo) da 2011-11-05 05:45:00 a 2011-11-09 14:00:00

3)  ->
Num video CON cicloni: 110, Num video SENZA cicloni: 1054	Totale video tiles: 1164
Bilanciamento video...		 video senza cicloni tenuti: 110

Creazione delle folder per i 220 video...	Salvati 1600 file - Erano già presenti 1920 file - File totali 3520
220 video per il periodo (effett

In [22]:
test_m = BuildDataset(type='SUPERVISED', args=args)
test_m.get_data_ready(tracks_df_test, input_dir, output_dir, csv_file="test_medicanes")

sorted_metadata_files num :  3952
Colonna label_24_00 aggiunta
1)  ->
Num video CON cicloni: 71, Num video SENZA cicloni: 877	Totale video tiles: 948
Bilanciamento video...		 video senza cicloni tenuti: 71

Creazione delle folder per i 142 video...	Salvati 2272 file - Erano già presenti 0 file - File totali 2272
142 video per il periodo (effettivo) da 2018-09-26 13:45:00 a 2018-09-29 23:00:00

2)  ->
Num video CON cicloni: 47, Num video SENZA cicloni: 649	Totale video tiles: 696
Bilanciamento video...		 video senza cicloni tenuti: 47

Creazione delle folder per i 94 video...	Salvati 1504 file - Erano già presenti 0 file - File totali 1504
94 video per il periodo (effettivo) da 2018-09-30 01:45:00 a 2018-10-02 14:00:00

3)  ->
Num video CON cicloni: 0, Num video SENZA cicloni: 162	Totale video tiles: 162
Bilanciamento video...		 video senza cicloni tenuti: 0
No video present for period: 2020-09-12 05:00:00 to 2020-09-12 21:00:00
4)  ->
Num video CON cicloni: 4, Num video SENZA cicloni: 